In [6]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import logging
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, naive_bayes, svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras import layers
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import warnings
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings("ignore")
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
data=pd.read_csv('outfit_combinations.csv')
d3=pd.read_csv('Full+data.csv')
d3=d3.dropna(axis=1,how='all')
data=data.merge(d3[['product_id','description']],on='product_id',how='inner')

In [8]:
data=data.dropna()
data=data.reset_index().drop(columns='index')

In [9]:
data['outfit_item_type'].unique()

array(['bottom', 'top', 'accessory1', 'shoe', 'onepiece', 'accessory2'],
      dtype=object)

In [5]:
#Preprocess data - columns outfit_item_type,brand,product_full_name,product_id
nltk.download('stopwords')
columns=['outfit_item_type','brand','product_full_name','product_id','description']
for j in columns:
    corpus = []
    for i in range(0,len(data)):
        new=re.sub('[^0-9a-zA-Z]', ' ',str(data.loc[i,j]))
        p=[]
        for k in new.split():
            separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
            new = separated.lower()
            p.append(new)
        ls=WordNetLemmatizer()
        new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
        new = ' '.join(new)
        data.loc[i,j]=new

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyboardInterrupt: 

In [6]:
data.isna().sum()

outfit_id            0
product_id           0
outfit_item_type     0
brand                0
product_full_name    0
description          0
dtype: int64

In [11]:
#combining info from all four columns into one before vectorization
data['all_info']=data['product_id']+' '+data['brand']+' '+data['outfit_item_type']+' '+data['product_full_name']+" "+data['description']

In [8]:
data.head()

,outfit_id,product_id,outfit_item_type,brand,product_full_name,description,all_info
0,01DDBHC62ES5K80P0KYJ56AM2T,01dmbryva2p5h24wk0htk4r0a1,bottom,eileen fisher,slim knit skirt,nice skirt,01dmbryva2p5h24wk0htk4r0a1 eileen fisher botto...
1,01DMHCX50CFX5YNG99F3Y65GQW,01dmbryva2p5h24wk0htk4r0a1,bottom,eileen fisher,slim knit skirt,nice skirt,01dmbryva2p5h24wk0htk4r0a1 eileen fisher botto...
2,01DDBHC62ES5K80P0KYJ56AM2T,01dmbryva2pepwftt7rmp5aa1t,top,eileen fisher,rib mock neck tank,nice tank,01dmbryva2pepwftt7rmp5aa1t eileen fisher top r...
3,01DMHCX50CFX5YNG99F3Y65GQW,01dmbryva2pepwftt7rmp5aa1t,top,eileen fisher,rib mock neck tank,nice tank,01dmbryva2pepwftt7rmp5aa1t eileen fisher top r...
4,01DDBHC62ES5K80P0KYJ56AM2T,01dmbryva2s5t9w793f4cy41he,accessory1,kate spade new york,medium margaux leather satchel,nice bag,01dmbryva2s5t9w793f4cy41he kate spade new york...


# TFIDF - COSINE

In [9]:
#TFIDF Vectorize three columns
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['all_info'])
terms = vectorizer.get_feature_names()
#c=pd.DataFrame(p.toarray().transpose(),index=terms)

In [10]:
#sample query
query='shoe: Penelope Mid Cap Toe Pump (01DMBRYVA2ZFDYRYY5TRQZJTAD)'

In [11]:
#Clean query
#Preprocess data - columns outfit_item_type,brand,product_full_name
nltk.download('stopwords')
new=re.sub('[^a-zA-Z0-9]', ' ',str(query))
p=[]
for k in new.split():
    separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
    new = separated.lower()
    p.append(new)
ls=WordNetLemmatizer()
new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
new = ' '.join(new)
query=new

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
#cleaned query
query

'shoe penelope mid cap toe pump 01dmbryva2zfdyryy5trqzjtad'

In [13]:
#TFIDF Query
Q = vectorizer.transform([query])
terms = vectorizer.get_feature_names()
#Q=pd.DataFrame(p.toarray().transpose(),index=terms)

In [14]:
#Find similarity with all documents
results=cosine_similarity(X,Q).reshape((-1,1))

In [15]:
#Recommended outfit based upon row which had maximum cosine similiarity
data[data['outfit_id']==data.iloc[np.argmax(results),0]]

,outfit_id,product_id,outfit_item_type,brand,product_full_name,description,all_info
0,01DDBHC62ES5K80P0KYJ56AM2T,01dmbryva2p5h24wk0htk4r0a1,bottom,eileen fisher,slim knit skirt,nice skirt,01dmbryva2p5h24wk0htk4r0a1 eileen fisher botto...
2,01DDBHC62ES5K80P0KYJ56AM2T,01dmbryva2pepwftt7rmp5aa1t,top,eileen fisher,rib mock neck tank,nice tank,01dmbryva2pepwftt7rmp5aa1t eileen fisher top r...
4,01DDBHC62ES5K80P0KYJ56AM2T,01dmbryva2s5t9w793f4cy41he,accessory1,kate spade new york,medium margaux leather satchel,nice bag,01dmbryva2s5t9w793f4cy41he kate spade new york...
6,01DDBHC62ES5K80P0KYJ56AM2T,01dmbryva2zfdyryy5trqzjtbd,shoe,tory burch,penelope mid cap toe pump,nice shoe,01dmbryva2zfdyryy5trqzjtbd tory burch shoe pen...


In [17]:
#Find max of an array full of cosine similarity numbers
import heapq
k=list(data.iloc[heapq.nlargest(1,range(len(results)),results.__getitem__),0])

In [18]:
#make a dataframe for recommendations
f=pd.DataFrame()
for i in k:
    for j in range(0,len(data)):
        l=[]
        if data.loc[j,'outfit_id']==i:
            l.append(list(data.iloc[j,[1,2,3,4]].values))
            if l!=[]:
                f=f.append(l)

In [19]:
f.columns=['product_id','outfit_type','brand','product_info']

In [20]:
f #Table of recommendations

,product_id,outfit_type,brand,product_info
0,01dmbryva2p5h24wk0htk4r0a1,bottom,eileen fisher,slim knit skirt
0,01dmbryva2pepwftt7rmp5aa1t,top,eileen fisher,rib mock neck tank
0,01dmbryva2s5t9w793f4cy41he,accessory1,kate spade new york,medium margaux leather satchel
0,01dmbryva2zfdyryy5trqzjtbd,shoe,tory burch,penelope mid cap toe pump


# Doc2 VecCosine

In [21]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data['all_info'])]
model = Doc2Vec(documents, vector_size=50, window=20, min_count=1, workers=3,negative=10,alpha=0.025)

In [22]:
v2=model.infer_vector([query]).reshape((1,-1))
a=0
for i in range(0,len(data)):
    v1=model.infer_vector([data.loc[i,'all_info']]).reshape((1,-1))
    if cosine_similarity(v1,v2)>a:
        a=cosine_similarity(v1, v2)
        p=i
data[data['outfit_id']==data.iloc[p,0]]
#p has the index of maximum similairity

,outfit_id,product_id,outfit_item_type,brand,product_full_name,description,all_info
4338,01DVVBRQ7M32WCCAD47WRTEFFZ,01drzjxqx38z9zy07tw52b39b2,shoe,rag bone,fei mule,almond toed mule contrasting wooden stacked he...,01drzjxqx38z9zy07tw52b39b2 rag bone shoe fei m...
4347,01DVVBRQ7M32WCCAD47WRTEFFZ,01dt0dm6nx4zfpgp2zcadmkvqw,accessory2,theory,double faced overlay coat,minimal overcoat luxe wool cashmere exudes ref...,01dt0dm6nx4zfpgp2zcadmkvqw theory accessory2 d...
4365,01DVVBRQ7M32WCCAD47WRTEFFZ,01dt50xz89mhr0atrcpjgbnr3e,top,la ligne,striped cashmere sweater,beige ivory cashmere slip 100 cashmere hand wa...,01dt50xz89mhr0atrcpjgbnr3e la ligne top stripe...
4371,01DVVBRQ7M32WCCAD47WRTEFFZ,01dt516tgq4efywxgkv2er5wra,bottom,j brand,joan cropped cotton blend corduroy wide leg pant,black cotton blend corduroy button concealed z...,01dt516tgq4efywxgkv2er5wra j brand bottom joan...
4383,01DVVBRQ7M32WCCAD47WRTEFFZ,01dvvbr6208whc8qx2e2cv411m,accessory1,prada,belle small leather shoulder bag,white leather calf tab fastening front flap co...,01dvvbr6208whc8qx2e2cv411m prada accessory1 be...


# Fuzzy Matching

In [24]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [25]:
query="slim fitting, straight leg pant with a center back zipper and slightly cropped leg and slightly cropped leg- BRAND: Reformation"

In [26]:
#Clean query
#Preprocess data - columns outfit_item_type,brand,product_full_name
nltk.download('stopwords')
new=re.sub('[^a-zA-Z]', ' ',str(query))
p=[]
for k in new.split():
    separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
    new = separated.lower()
    p.append(new)
ls=WordNetLemmatizer()
new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
new = ' '.join(new)
query=new

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
query

'slim fitting straight leg pant center back zipper slightly cropped leg slightly cropped leg brand reformation'

In [28]:
#Create dataframe using row that had maximum similarity
b=0
for i in range(0,len(data)):
    a=fuzz.partial_ratio(data.loc[i,'all_info'],query)
    if a>b:
        b=a
        p=i
f=data[data['outfit_id']==data.iloc[p,0]]
pd.DataFrame(f[['product_id','outfit_item_type','brand','product_full_name']])

,product_id,outfit_item_type,brand,product_full_name
35,01dpehs0xh9pdd1gh5ze4p43a2,accessory1,sole society,cassi belt bag
37,01dpkmh0d252jkmaa27mfct5gm,bottom,reformation,marlon pant
38,01dpkn20q3j0be3cs896dqb6er,top,reformation,jane sweater
39,01dpknhqdg6gptkv97cfqrjdhe,shoe,reformation,giulia satin heel


# Functions - TFIDF

In [29]:
#Preprocess data - columns outfit_item_type,brand,product_full_name
nltk.download('stopwords')
from nltk.corpus import stopwords
def recommend_items(data,query):
    columns=['outfit_item_type','brand','product_full_name','description']
    for j in columns:
        for i in range(0,len(data)):
            new=re.sub('[^a-zA-Z0-9]', ' ',str(data.loc[i,j]))
            p=[]
            for k in new.split():
                separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
                new = separated.lower()
                p.append(new)
            ls=WordNetLemmatizer()
            new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
            new = ' '.join(new)
            data.loc[i,j]=new
    data['all_info']=data['product_id']+' '+data['brand']+' '+data['outfit_item_type']+' '+data['product_full_name']+" "+data['description']
        #TFIDF Vectorize three columns
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data['all_info'])
    terms = vectorizer.get_feature_names()
    #c=pd.DataFrame(p.toarray().transpose(),index=terms)
    #Clean query
    #Preprocess data - columns outfit_item_type,brand,product_full_name
    nltk.download('stopwords')
    new=re.sub('[^0-9a-zA-Z]', ' ',str(query))
    p=[]
    for k in new.split():
        separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
        new = separated.lower()
        p.append(new)
    ls=WordNetLemmatizer()
    new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
    new = ' '.join(new)
    query=new

    #TFIDF Query
    Q = vectorizer.transform([query])
    terms = vectorizer.get_feature_names()
    #Q=pd.DataFrame(p.toarray().transpose(),index=terms)
    
    from sklearn.metrics.pairwise import cosine_similarity
    #Find similarity with all documents
    results=cosine_similarity(X,Q).reshape((-1,1))
    
    import heapq
    k=list(data.iloc[heapq.nlargest(1,range(len(results)),results.__getitem__),0])
    for i in k:
        f=pd.DataFrame()
        for j in range(0,len(data)):
            l=[]
            if data.loc[j,'outfit_id']==i:
                l.append(list(data.iloc[j,[1,2,4]].values))
                if l!=[]:
                    f=f.append(l)
        f.columns=['product_id','brand','item']
        f['product_id']=f['product_id'].str.upper()
        return f

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
recommend_items(data,query)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,product_id,brand,item
0,01DPEHS0XH9PDD1GH5ZE4P43A2,accessory1,cassi belt bag
0,01DPKMH0D252JKMAA27MFCT5GM,bottom,marlon pant
0,01DPKN20Q3J0BE3CS896DQB6ER,top,jane sweater
0,01DPKNHQDG6GPTKV97CFQRJDHE,shoe,giulia satin heel


In [14]:
query='bottom: DESCRIPTION: slim fitting, straight leg pant with a center back zipper and slightly cropped leg – BRAND: Reformation'

In [32]:
#query='Sexy silky, a-line mini skirt zipper Benson skirt'

In [33]:
#data

# Function Fuzzy

In [73]:
def recommend_fuzz(data,query):
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    columns=['outfit_item_type','brand','product_full_name','description']
    for j in columns:
        for i in range(0,len(data)):
            new=re.sub('[^a-zA-Z0-9]', ' ',str(data.loc[i,j]))
            p=[]
            for k in new.split():
                separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
                new = separated.lower()
                p.append(new)
            ls=WordNetLemmatizer()
            new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
            new = ' '.join(new)
            data.loc[i,j]=new
    data['all_info']=data['product_id']+' '+data['brand']+' '+data['outfit_item_type']+' '+data['product_full_name']+' '+data['description']
        #TFIDF Vectorize three columns
    #Clean query
    #Preprocess data - columns outfit_item_type,brand,product_full_name
    
    new=re.sub('[^a-zA-Z]', ' ',str(query))
    p=[]
    for k in new.split():
        separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
        new = separated.lower()
        p.append(new)
    ls=WordNetLemmatizer()
    new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
    new = ' '.join(new)
    query=new
    b=0
    for i in range(0,len(data)):
        a=fuzz.partial_ratio(data.loc[i,'all_info'],query)
        if a>b:
            b=a
            p=i
    CD=data[data['outfit_id']==data.iloc[p,0]]
    CD['product_id']=CD['product_id'].str.upper()
    
    return CD.iloc[:,[1,2,4]]

In [35]:
recommend_fuz(data,query)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,product_id,outfit_item_type,product_full_name
35,01DPEHS0XH9PDD1GH5ZE4P43A2,accessory1,cassi belt bag
37,01DPKMH0D252JKMAA27MFCT5GM,bottom,marlon pant
38,01DPKN20Q3J0BE3CS896DQB6ER,top,jane sweater
39,01DPKNHQDG6GPTKV97CFQRJDHE,shoe,giulia satin heel


# Universal Sentence Encoder

In [12]:
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
V1 = embed(data['all_info'])

In [15]:
Q = embed([query])

In [16]:
results=cosine_similarity(V1,Q)

In [17]:
import heapq
k=list(data.iloc[heapq.nlargest(2,range(len(results)),results.__getitem__),0])

In [22]:
f=pd.DataFrame()
for i in k:
    for j in range(0,len(data)):
        l=[]
        if data.loc[j,'outfit_id']==i:
            l.append(list(data.iloc[j,[1,2,4]].values))
            if l!=[]:
                f=f.append(l)
    print(f)

                            0           1                  2
0  01DPEHS0XH9PDD1GH5ZE4P43A2  accessory1     Cassi Belt Bag
0  01DPKMH0D252JKMAA27MFCT5GM      bottom        Marlon Pant
0  01DPKN20Q3J0BE3CS896DQB6ER         top       Jane Sweater
0  01DPKNHQDG6GPTKV97CFQRJDHE        shoe  Giulia Satin Heel
                            0           1  \
0  01DPEHS0XH9PDD1GH5ZE4P43A2  accessory1   
0  01DPKMH0D252JKMAA27MFCT5GM      bottom   
0  01DPKN20Q3J0BE3CS896DQB6ER         top   
0  01DPKNHQDG6GPTKV97CFQRJDHE        shoe   
0  01DT0C8FVZJ77EEMXPSKD0VG3A      bottom   
0  01DT0DHR8X5GZYR1KVBWAT3END  accessory2   
0  01DT0DM2ZFCWSBKZ19W8WGPBQN  accessory1   
0  01DT51234VHAHGPTR89SZJ50V0        shoe   
0  01DVP9VHNTNDG4BX0SSHSTCZG8         top   

                                                   2  
0                                     Cassi Belt Bag  
0                                        Marlon Pant  
0                                       Jane Sweater  
0                       

In [19]:
k

['01DQ8ME3M3QS9MQGZCQHXDHE1R', '01DVPABBBDK2JJ2JGSKXNKZ13M']

In [20]:
f

,0,1,2
0,01DPEHS0XH9PDD1GH5ZE4P43A2,accessory1,Cassi Belt Bag
0,01DPKMH0D252JKMAA27MFCT5GM,bottom,Marlon Pant
0,01DPKN20Q3J0BE3CS896DQB6ER,top,Jane Sweater
0,01DPKNHQDG6GPTKV97CFQRJDHE,shoe,Giulia Satin Heel
0,01DT0C8FVZJ77EEMXPSKD0VG3A,bottom,Anson Crepe Slim Fit Pants
0,01DT0DHR8X5GZYR1KVBWAT3END,accessory2,Madison Single-Breasted Leather Jacket
0,01DT0DM2ZFCWSBKZ19W8WGPBQN,accessory1,Small Monogram Matelassé Metallic Leather Wall...
0,01DT51234VHAHGPTR89SZJ50V0,shoe,Andrea 85 zebra-print calf hair and leather mules
0,01DVP9VHNTNDG4BX0SSHSTCZG8,top,Athena lace-up crepe blouse


# Function for Universal Sentence encoder

In [23]:
#Preprocess data - columns outfit_item_type,brand,product_full_name
def recommend_USE(data,query):
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    columns=['outfit_item_type','brand','product_full_name','description']
    for j in columns:
        for i in range(0,len(data)):
            new=re.sub('[^a-zA-Z0-9]', ' ',str(data.loc[i,j]))
            p=[]
            for k in new.split():
                separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
                new = separated.lower()
                p.append(new)
            ls=WordNetLemmatizer()
            new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
            new = ' '.join(new)
            data.loc[i,j]=new
    data['all_info']=data['product_id']+' '+data['brand']+' '+data['outfit_item_type']+' '+data['product_full_name']+" "+data['description']
    #Universal Sentence encoder Vectorize four columns
    import tensorflow as tf
    import tensorflow_hub as hub
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    V1 = embed(data['all_info'])
    #Preprocess data - columns outfit_item_type,brand,product_full_name
    nltk.download('stopwords')
    new=re.sub('[^0-9a-zA-Z]', ' ',str(query))
    p=[]
    for k in new.split():
        separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
        new = separated.lower()
        p.append(new)
    ls=WordNetLemmatizer()
    new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
    new = ' '.join(new)
    query=new

    #Universal Sentence Vectorize Query
    Q = embed([query])
    from sklearn.metrics.pairwise import cosine_similarity
    #Find similarity with all documents
    results=cosine_similarity(V1,Q)
    
    import heapq
    k=list(data.iloc[heapq.nlargest(2,range(len(results)),results.__getitem__),0])
    for i in k:
        f=pd.DataFrame()
        for j in range(0,len(data)):
            l=[]
            if data.loc[j,'outfit_id']==i:
                l.append(list(data.iloc[j,[1,2,4]].values))
                if l!=[]:
                    f=f.append(l)
        f.columns=['product_id','brand','item','desc']
        f['product_id']=f['product_id'].str.upper()
        return f

In [ ]:
recommend_USE(data,query)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
query='01DTJBR06HEYC2WWGJX1RKBZA5 shoe meta boot'

In [76]:
pd.set_option('display.max_colwidth', -1)
recommend_USE(data,query)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,product_id,brand,item,desc
0,01DT2D39AP6T64ZA88WEPQ1C3K,accessory1,miranda croc effect leather shoulder bag,bag brand far launched handbag business enormous success megahit footwear miranda style crafted trend mini silhouette croc effect leather nutella hue front flap open moderately sized interior fit cell beauty essential
0,01DT51AFKN7GGVG7EFW0ZGF3PE,accessory2,ribbed knit wrap cardigan,brown ribbed knit tie front 40 wool 40 acrylic 20 nylon dry clean imported
0,01DT8ND4QVV25CDSARCDFCWZ11,bottom,alloisa belted striped crepe slim leg pant,equipment incorporates sleek tailoring fall 19 lineup label alloisa pant cut lightweight crepe patterned classic pinstripe tapered slim leg shape coordinating belt emphasizes high rise waist style tucked blouse structured blazer
0,01DTJBEY26533CKXMXH26MN2SH,top,metallic wool blend turtleneck,brunello cucinelli short sleeved turtleneck spun blend mohair virgin wool cashmere metallic element shimmer lightweight feel semi sheer body make versatile enough wear throughout year styled different way
0,01DTJBR06HEYC2WWGJX1RKBZA4,shoe,meta snakeskin embossed leather ankle boot,sleek point toe ankle boot crafted supple leather luxe snakeskin embossed finish


In [74]:
recommend_fuzz(data,query)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,product_id,outfit_item_type,product_full_name
697,01DT2D39AP6T64ZA88WEPQ1C3K,accessory1,miranda croc effect leather shoulder bag
716,01DT51AFKN7GGVG7EFW0ZGF3PE,accessory2,ribbed knit wrap cardigan
753,01DT8ND4QVV25CDSARCDFCWZ11,bottom,alloisa belted striped crepe slim leg pant
760,01DTJBEY26533CKXMXH26MN2SH,top,metallic wool blend turtleneck
773,01DTJBR06HEYC2WWGJX1RKBZA4,shoe,meta snakeskin embossed leather ankle boot
